In [2]:
# Libraries
import pandas as pd

In [3]:
# Let's come up with a definition for a 'valid' time element record
# Before that, let's read in the dataset

data = pd.read_excel("C:\\Users\\shout\\Desktop\\Shou Teck Time Elements.xlsx")
data.head()

,PortCallID,CargoID,TimeEventID,TimeEventFunc,TimeEventName,TEValueDate,TEValueTime
0,5250050329,NaN,553000168.0,Bunkering,Hoses Connected,2019-10-02,14:04:00
1,5250050329,NaN,553000169.0,Bunkering,Hoses Disconnected,2019-10-02,18:05:00
2,5250050329,NaN,553000023.0,Bunkering,Barge Away,2019-10-02,18:55:00
3,5204050120,NaN,553000215.0,Arrival,Notice of Readiness Accepted,2019-09-23,08:40:00
4,5204050120,NaN,553000159.0,Arrival,Free Pratique,2019-09-23,08:35:00


In [4]:
data.shape

(199906, 7)

In [5]:
# 1st criterion that qualifies a time element record as 'invalid' is that all the columns other than "PortCallID" are NULL 
# There are no useful insights that you can get out of it
# Let's see how many of these records are there in the dataset
# One thing I realised is that all of these records must have "TimeEventID" as NULL

null_data = data[data["TimeEventID"].isna()]

In [6]:
# That is 30% of the dataset consisting of 'invalid' time element records
null_data.shape

(61383, 7)

In [7]:
# Since they are useless we can remove them

data = data.dropna(subset = ["TimeEventID"]) 
data.shape

(138523, 7)

In [8]:
# 2nd criterion that qualifies a time element record as 'invalid' is that both "TEValueDate" and "TEValueTime" are NULL
# Associated with every time event has to have a starting date and the duration of it
# Let's see how many of these records are there in the dataset

null_data = data[data["TEValueDate"].isna() & data["TEValueTime"].isna()]
null_data.head()

,PortCallID,CargoID,TimeEventID,TimeEventFunc,TimeEventName,TEValueDate,TEValueTime
42,5257049296,NaN,553000007.0,Arrival,Anchored,NaT,NaN
48,5257049296,a0J6F000016jsdZUAQ,553000066.0,Cargo,Discharging Commenced,NaT,NaN
49,5257049296,a0J6F000016jsdZUAQ,553000094.0,Cargo,Discharging Completed,NaT,NaN
54,5257049296,NaN,553000210.0,Departure,Next Port ETA,NaT,NaN
73,5226047284,NaN,553000017.0,Arrival,Arrived,NaT,NaN


In [9]:
# That is 27% of the dataset consisting of 'invalid' time element records
null_data.shape

(55213, 7)

In [10]:
# Since they are useless we can remove them

data = data.dropna(subset = ["TEValueDate", "TEValueTime"]) 
data.shape

(82805, 7)

In [11]:
# From here onwards, the 'invalid' time element records won't be as generic as before
# I will take a look at specifically "TimeEventFunc" = Cargo and see if there isn't a CargoID associated with it 
# Seems 'valid'
null_data = data[(data["CargoID"].isna()) & (data["TimeEventFunc"] == "Cargo")]
null_data.head()

,PortCallID,CargoID,TimeEventID,TimeEventFunc,TimeEventName,TEValueDate,TEValueTime


In [12]:
# Vice-versa, check if there are "TimeEventFunc" != Cargo that has a CargoID associated with it
# Seems 'valid'
null_data = data[(data["CargoID"].isna() == False) & (data["TimeEventFunc"] != "Cargo")]
null_data.head()

,PortCallID,CargoID,TimeEventID,TimeEventFunc,TimeEventName,TEValueDate,TEValueTime


In [13]:
# Here, I'm checking if the for every logical operation that require a completion and a commencement 
# Such as "TimeEventFunc" = Cargo where sometimes you load/discharge and these operations
# come in a pair of (completion, commencement)
# Indeed, there are some inconsistencies
data[data["TimeEventFunc"] == "Cargo"]["TimeEventName"].value_counts()

Loading Completed                1577
Loading Commenced                1571
Discharging Commenced            1252
Discharging Completed            1238
Calculation Completed             425
                                 ... 
Shore Line Sample Commenced         1
Hatch Closing Commenced             1
Gas Free Inspection Commenced       1
Inerting Commenced                  1
Shore Line Sample Completed         1
Name: TimeEventName, Length: 119, dtype: int64

In [14]:
# For every pair of (completion, commencement) we can find here, we can always remove the inconsistent time element records 
# that has either completion/commencement but lack its counterpart
data[data["TimeEventFunc"] == "Cargo"].TimeEventName.unique()

array(['Loading Commenced', 'Loading Completed',
       'Est Time of Cargo Completion', 'Arm(s) Connection Completed',
       'Tank Inspection Completed', 'Calculation Completed',
       'Documents On Board (Outwards)', 'Arm(s) Disconnection Completed',
       'Pilot On Board', 'Hoses Connected', 'Hoses Disconnected',
       'Discharging Commenced', 'Discharging Completed',
       'Gas Free Inspection Completed', 'Tank Inspection Commenced',
       'Hose Connection Completed', 'Hose Disconnection Completed',
       'Notice of Readiness Accepted', 'Arm(s) Connection Commenced',
       'Safety Meeting Commenced', 'Safety Meeting Completed',
       'Ullaging Commenced', 'Ullaging Completed', 'Surveyor On Board',
       'Discharge Master On Board', 'Sampling Commenced',
       'Sampling Completed', 'Calculations Commenced', 'Ramp Up',
       'Ramp Down', 'Last Line Off', 'Final Draft Survey Completed',
       'Gangway Down', 'Loading Master On Board',
       'Notice of Readiness Tendered',

In [15]:
# 3rd criterion that qualifies a time element record as 'invalid' is that any "TimeEventName" operation that should come as 
# a pair did not come as a pair
# To achieve this, I may have to loop through the rows of the dataset which is an anti-pattern so I'm not so sure whats 
# the efficient way to do this

In [16]:
# The current size of the dataset with 'valid' time element records
# Assuming that I have eliminated the invalid records using the 3rd criterion, the dataset size will be < 82805
# That is a > 58% reduction is the dataset size
data.shape

(82805, 7)

In [17]:
loading_commence = data[(data["TimeEventFunc"] == "Cargo") & (data["TimeEventName"] == "Loading Commenced")]
loading_complete = data[(data["TimeEventFunc"] == "Cargo") & (data["TimeEventName"] == "Loading Completed")]
loading_commence.head()

,PortCallID,CargoID,TimeEventID,TimeEventFunc,TimeEventName,TEValueDate,TEValueTime
21,5272049254,a0J6F000016jsVuUAI,553000068.0,Cargo,Loading Commenced,2019-09-30,08:30:00
22,5272049254,a0J6F000016jsVvUAI,553000068.0,Cargo,Loading Commenced,2019-09-29,09:35:00
23,5272049254,a0J6F000016jsVwUAI,553000068.0,Cargo,Loading Commenced,2019-09-29,19:15:00
353,5328050339,a0J6F000018FfzoUAC,553000068.0,Cargo,Loading Commenced,2019-10-06,11:24:00
800,5346050359,a0J6F000018Fg2mUAC,553000068.0,Cargo,Loading Commenced,2019-10-03,14:00:00


In [18]:
loading_complete.head()

,PortCallID,CargoID,TimeEventID,TimeEventFunc,TimeEventName,TEValueDate,TEValueTime
24,5272049254,a0J6F000016jsVuUAI,553000104.0,Cargo,Loading Completed,2019-09-30,08:45:00
25,5272049254,a0J6F000016jsVvUAI,553000104.0,Cargo,Loading Completed,2019-09-29,11:00:00
26,5272049254,a0J6F000016jsVwUAI,553000104.0,Cargo,Loading Completed,2019-09-29,20:20:00
354,5328050339,a0J6F000018FfzoUAC,553000104.0,Cargo,Loading Completed,2019-10-08,03:28:00
801,5346050359,a0J6F000018Fg2mUAC,553000104.0,Cargo,Loading Completed,2019-10-03,19:30:00


In [23]:
# (Assumption) Going with the logic that commencement comes together with completion
# The average will be an estimate as well as there isn't a 1-1 correspondence for every commencement and completion
# Since #LoadingCompletion exceeds #LoadingCommencement, we will treat some of the LoadingCompletion as a pair for excess
# LoadingCompletion records that do not have a pair yet

# (1577 + 1571) / 2 = 1574 
# There are 1574 pairs of (commencement, completion)

In [24]:
# Lets convert "TEValueTime" to seconds first for easy computation
loading_complete_seconds = (pd.to_timedelta(loading_complete['TEValueTime'].astype(str)).dt.seconds).to_frame()
loading_complete_seconds.head()

,TEValueTime
24,31500
25,39600
26,73200
354,12480
801,70200


In [26]:
# Similarly, for loading commencement
loading_commence_seconds = (pd.to_timedelta(loading_commence['TEValueTime'].astype(str)).dt.seconds).to_frame()
loading_commence_seconds.head()

,TEValueTime
21,30600
22,34500
23,69300
353,41040
800,50400


In [30]:
# Lets sum the values in both tables and average it 
avg_loading = (loading_commence_seconds['TEValueTime'].sum() + loading_complete_seconds['TEValueTime'].sum())
avg_loading = avg_loading / ((loading_commence_seconds.shape)[0] + (loading_complete_seconds.shape)[0])
avg_loading

44261.62642947903

In [31]:
# I'm going to repeat the process for discharging

discharging_commence = data[(data["TimeEventFunc"] == "Cargo") & (data["TimeEventName"] == "Discharging Commenced")]
discharging_complete = data[(data["TimeEventFunc"] == "Cargo") & (data["TimeEventName"] == "Discharging Completed")]

discharging_complete_seconds = (pd.to_timedelta(discharging_complete['TEValueTime'].astype(str)).dt.seconds).to_frame()
discharging_commence_seconds = (pd.to_timedelta(discharging_commence['TEValueTime'].astype(str)).dt.seconds).to_frame()

avg_discharging = (discharging_commence_seconds['TEValueTime'].sum() + discharging_complete_seconds['TEValueTime'].sum())
avg_discharging = avg_discharging / ((discharging_commence_seconds.shape)[0] + (discharging_complete_seconds.shape)[0])
avg_discharging

45449.8313253012